In [2]:
import tensorflow
import tensorflow.keras
tensorflow.keras.__version__

'2.2.4-tf'

In [3]:
import os, shutil

In [47]:
# 소규모 데이터셋을 저장할 디렉터리
base_dir = './datasets/small'

# 훈련, 검증, 테스트 분할을 위한 디렉터리
train_dir = './datasets/small/train'
validation_dir = './datasets/small/validation'
test_dir = './datasets/small/test'


# 훈련용 고양이 사진 디렉터리
train_cats_dir = './datasets/small/train/cats'

# 훈련용 강아지 사진 디렉터리
train_dogs_dir = './datasets/small/train/dogs'

# 검증용 고양이 사진 디렉터리
validation_cats_dir = './datasets/small/validation/cats'

# 검증용 강아지 사진 디렉터리
validation_dogs_dir = './datasets/small/validation/dogs'

# 테스트용 고양이 사진 디렉터리
test_cats_dir = './datasets/small/test/cats'

# 테스트용 강아지 사진 디렉터리
test_dogs_dir = './datasets/small/test/dogs'

In [48]:
print('훈련용 고양이 이미지 전체 개수:', len(os.listdir(train_cats_dir)))

훈련용 고양이 이미지 전체 개수: 101


In [49]:
print('훈련용 강아지 이미지 전체 개수:', len(os.listdir(train_dogs_dir)))

훈련용 강아지 이미지 전체 개수: 101


In [50]:
print('검증용 고양이 이미지 전체 개수:', len(os.listdir(validation_cats_dir)))

검증용 고양이 이미지 전체 개수: 51


In [51]:
print('검증용 강아지 이미지 전체 개수:', len(os.listdir(validation_dogs_dir)))

검증용 강아지 이미지 전체 개수: 50


In [52]:
print('테스트용 고양이 이미지 전체 개수:', len(os.listdir(test_cats_dir)))

테스트용 고양이 이미지 전체 개수: 50


In [53]:
print('테스트용 강아지 이미지 전체 개수:', len(os.listdir(test_dogs_dir)))

테스트용 강아지 이미지 전체 개수: 50


In [54]:
from tensorflow.keras import layers
from tensorflow.keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [55]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 15, 15, 128)      

In [56]:
from tensorflow.keras import optimizers

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

In [57]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 모든 이미지를 1/255로 스케일을 조정합니다
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # 타깃 디렉터리
        train_dir,
        # 모든 이미지를 150 × 150 크기로 바꿉니다
        target_size=(150, 150),
        batch_size=20,
        # binary_crossentropy 손실을 사용하기 때문에 이진 레이블이 필요합니다
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=50,
        class_mode='binary')

Found 202 images belonging to 2 classes.
Found 101 images belonging to 2 classes.


In [58]:
for data_batch, labels_batch in train_generator:
    print('배치 데이터 크기:', data_batch.shape)
    print('배치 레이블 크기:', labels_batch.shape)
    break

배치 데이터 크기: (20, 150, 150, 3)
배치 레이블 크기: (20,)


In [59]:
history = model.fit_generator(
      train_generator,
      #steps_per_epoch=10,
      epochs=20,
      validation_data=validation_generator,
      #validation_steps=50
      )

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 11 steps, validate for 3 steps
Epoch 1/20
11/11 [==============================] - 6s 526ms/step - loss: 0.7182 - acc: 0.4703 - val_loss: 0.6723 - val_acc: 0.5050
Epoch 2/20
11/11 [==============================] - 5s 467ms/step - loss: 0.6856 - acc: 0.5545 - val_loss: 0.7427 - val_acc: 0.5050
Epoch 3/20
11/11 [==============================] - 5s 464ms/step - loss: 0.6824 - acc: 0.5050 - val_loss: 0.6887 - val_acc: 0.5644
Epoch 4/20
11/11 [==============================] - 5s 483ms/step - loss: 0.6656 - acc: 0.6337 - val_loss: 0.5771 - val_acc: 0.4950
Epoch 5/20
11/11 [==============================] - 5s 486ms/step - loss: 0.6492 - acc: 0.6188 - val_loss: 0.6266 - val_acc: 0.5446
Epoch 6/20
11/11 [==============================] - 5s 486ms/step - loss: 0.6309 - acc: 0.6881 - val_loss: 0.6764 - val_acc: 0.5545
Epoch 7/20
11/11 [==============================] - 5s 494ms/step - loss: 0.5901 - acc: 0.6881 - val_loss: 0.6031 - v

훈련이 끝나면 항상 모델을 저장하는 것이 좋은 습관입니다:

In [60]:
model.save('cats_and_dogs_small_1.h5')

In [61]:
from tensorflow.keras.models import load_model

model=load_model('cats_and_dogs_small_1.h5')

In [62]:
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        batch_size=50,
        class_mode='binary')

Found 100 images belonging to 2 classes.


In [63]:
print("-- Evaluate --")
scores = model.evaluate(test_generator)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

-- Evaluate --
  ...
    to  
  ['...']
2/2 [==============================] - 1s 424ms/step - loss: 0.7927 - acc: 0.5900
acc: 59.00%


In [65]:
import numpy as np

output = model.predict_generator(test_generator, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_generator.class_indices)
print(output)

{'cats': 0, 'dogs': 1}
[[0.324]
 [0.385]
 [0.034]
 [0.496]
 [0.305]
 [0.682]
 [0.715]
 [0.323]
 [0.833]
 [0.639]
 [0.478]
 [0.340]
 [0.207]
 [0.534]
 [0.433]
 [0.838]
 [0.363]
 [0.053]
 [0.982]
 [0.416]
 [0.737]
 [0.702]
 [0.151]
 [0.556]
 [0.617]
 [0.623]
 [0.204]
 [0.284]
 [0.959]
 [0.736]
 [0.397]
 [0.058]
 [0.128]
 [0.209]
 [0.571]
 [0.835]
 [0.266]
 [0.780]
 [0.053]
 [0.416]
 [0.225]
 [0.537]
 [0.210]
 [0.253]
 [0.714]
 [0.042]
 [0.414]
 [0.754]
 [0.709]
 [0.786]
 [0.774]
 [0.198]
 [0.811]
 [0.150]
 [0.646]
 [0.090]
 [0.216]
 [0.596]
 [0.795]
 [0.555]
 [0.198]
 [0.031]
 [0.531]
 [0.125]
 [0.484]
 [0.067]
 [0.764]
 [0.481]
 [0.818]
 [0.341]
 [0.504]
 [0.144]
 [0.273]
 [0.750]
 [0.131]
 [0.624]
 [0.363]
 [0.259]
 [0.164]
 [0.919]
 [0.272]
 [0.519]
 [0.425]
 [0.072]
 [0.184]
 [0.886]
 [0.357]
 [0.543]
 [0.326]
 [0.043]
 [0.499]
 [0.703]
 [0.868]
 [0.418]
 [0.318]
 [0.399]
 [0.558]
 [0.582]
 [0.654]
 [0.294]]
